In [4]:
import os
import numpy as np
import pandas as pd

from scipy.stats import norm
from scipy.optimize import minimize

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
# If you have a local module called paramsgen (and need it), uncomment:
# from paramsgen import *


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [6]:
def predict_efforts_with_model(subdf, params, fixed_sigma=10.0):
    """
    subdf: participant's DataFrame with columns at least:
       trial, threshold, reward
       (chosen_effort is optional if you only want the model's behavior).
    params = [alpha_r, alpha_e, kappa, beta]
    fixed_sigma: float
    
    Returns a new DataFrame with columns:
      trial, threshold, reward, predicted_effort
      plus optional columns R_hat, E_hat if you want to store them.
    """

    alpha_r, alpha_e, kappa, beta = params

    # Initialize hidden states
    R_hat = 3.0
    E_hat = 50.0

    # Sort by trial to ensure correct chronological order
    subdf = subdf.sort_values('trial', ascending=True)

    records = []

    for i, row in subdf.iterrows():
        threshold_t = row['threshold']
        reward_t    = row['reward']

        # Evaluate EU(e) for e=0..100
        e_values = np.arange(101)
        EU_list  = []
        for e in e_values:
            p_succ = norm.cdf((e - E_hat) / fixed_sigma)
            SV     = R_hat - kappa * e
            EU_list.append(SV * p_succ)

        EU_array = np.array(EU_list)
        # Softmax or Argmax approach:
        maxEU = np.max(EU_array)
        expEU = np.exp(beta * (EU_array - maxEU))
        probs = expEU / np.sum(expEU)
        predicted_e = np.random.choice(e_values, p=probs)
        # or for deterministic:
        # predicted_e = e_values[np.argmax(EU_array)]

        # (optional) outcome logic
        p_succ_obs = norm.cdf((predicted_e - E_hat)/fixed_sigma)
        outcome = 1 if (np.random.rand() < p_succ_obs) else 0
        received_reward = reward_t if outcome==1 else 0.0

        # Update learners
        R_hat += alpha_r*(received_reward - R_hat)
        E_hat += alpha_e*(threshold_t - E_hat)
        E_hat = np.clip(E_hat, 0, 100)

        records.append({
            'trial'           : row['trial'],
            'threshold'       : threshold_t,
            'reward'          : reward_t,
            'predicted_effort': predicted_e,
            'R_hat'           : R_hat,
            'E_hat'           : E_hat
        })

    return pd.DataFrame(records)


In [7]:
def plot_per_participant(big_dataframe, results_df, fixed_sigma=10.0):
    for idx, row in results_df.iterrows():
        pid     = row['id']
        alpha_r = row['alpha_r']
        alpha_e = row['alpha_e']
        kappa   = row['kappa']
        beta    = row['beta']

        # Extract that participant's data from big_dataframe
        subdf = big_dataframe[big_dataframe['id'] == pid].copy()
        subdf.sort_values('trial', ascending=True, inplace=True)

        # If the user data has 'chosen_effort', we can plot it
        if 'chosen_effort' not in subdf.columns:
            print(f"No chosen_effort column for participant {pid}, skipping plot.")
            continue

        # Use best-fit params to generate predicted efforts
        params = [alpha_r, alpha_e, kappa, beta]
        pred_df = predict_efforts_with_model(subdf, params, fixed_sigma=fixed_sigma)

        # Merge so we have trial, threshold, chosen_effort, predicted_effort all in one DataFrame
        merged = pd.merge(subdf, pred_df, on='trial', how='left')


        plt.figure(figsize=(8,5))
        plt.plot(merged['trial'], merged['threshold'], label='Threshold', color='blue')
        plt.plot(merged['trial'], merged['chosen_effort'], label='Actual Chosen Effort', color='red')
        plt.plot(merged['trial'], merged['predicted_effort'], label='Predicted Effort', color='green')

        plt.xlabel('Trial')
        plt.ylabel('Effort / Threshold')
        plt.title(f"Participant {pid} (alpha_r={alpha_r:.2f}, alpha_e={alpha_e:.2f}, kappa={kappa:.3f}, beta={beta:.1f})")
        plt.legend()
        plt.show()


In [8]:

###############################################################################
# 1) READING AND PREPARING THE DATA
###############################################################################
def read_csv_files(directory):
    """
    Reads all 'sub*.csv' files in the specified directory and returns a list of DataFrames.
    Skips any files that do not match that pattern.
    """
    csv_files = []
    for filename in os.listdir(directory):
        if filename.startswith("sub") and filename.endswith(".csv"):
            print(f"Reading CSV file: {filename}")
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            csv_files.append(df)
        else:
            print(f"Skipping file: {filename}")
    return csv_files


def map_prolific_ids(dataframe):
    """
    Maps each unique prolific_id to an integer in [1..100].
    Returns the modified DataFrame and the mapping dictionary.
    """
    unique_ids = dataframe['prolific_id'].unique()
    id_mapping = {pid: (i % 100) + 1 for i, pid in enumerate(unique_ids)}
    dataframe['prolific_id'] = dataframe['prolific_id'].map(id_mapping)
    return dataframe, id_mapping


def normalize_column_to_0_100(dataframe, column_name):
    """
    Rescales the indicated column to [0..100].
    """
    min_val = dataframe[column_name].min()
    max_val = dataframe[column_name].max()
    scaled = 100.0 * (dataframe[column_name] - min_val) / (max_val - min_val)
    return scaled


###############################################################################
# 2) LOAD THE DATA
###############################################################################
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, 'data')

# 2a) Load a special "task_data_nobehav.csv" if it exists
task_data_path = os.path.join(data_directory, 'task_data_nobehav.csv')
task_data = None
if os.path.exists(task_data_path):
    task_data = pd.read_csv(task_data_path)
    # Rename columns to match our modeling approach
    # (If you need 'reward' and 'threshold' in your simulation, do so here)
    task_data.rename(columns={'Points': 'reward', 'effLevel': 'threshold'}, inplace=True)
    # Keep only relevant columns
    task_data = task_data[['reward', 'threshold']]
    print("Loaded task_data_nobehav.csv (renamed to 'reward' and 'threshold').")

# 2b) Load participant CSV files
dataframes = read_csv_files(data_directory)

# Ensure task_data is not included accidentally in the dataframes list
if task_data is not None:
    dataframes = [df for df in dataframes if not df.equals(task_data)]

# Concatenate all participant DataFrames into one big DataFrame
big_dataframe = pd.concat(dataframes, ignore_index=True)
print(f"\nConcatenated {len(dataframes)} participant DataFrames into one: {len(big_dataframe)} rows total.")

# 2c) Map prolific IDs to numeric
big_dataframe, id_mapping_df = map_prolific_ids(big_dataframe)

# 2d) Drop rows where threshold column (effLevel) is NaN
if 'effLevel' in big_dataframe.columns:
    big_dataframe = big_dataframe.dropna(subset=['effLevel'])

# Also drop rows where trials.thisN or trials.thisTrialN is NaN
if 'trials.thisN' in big_dataframe.columns:
    big_dataframe = big_dataframe.dropna(subset=['trials.thisN'])
if 'trials.thisTrialN' in big_dataframe.columns:
    big_dataframe = big_dataframe.dropna(subset=['trials.thisTrialN'])

# Drop empty columns
big_dataframe = big_dataframe.dropna(axis=1, how='all')

# Rename columns to consistent names
rename_map = {
    'Points': 'reward',
    'effLevel': 'threshold',
    'prolific_id': 'id',
    'percentage of bar reached': 'chosen_effort',
    'trials.thisTrialN': 'trial'  # or 'trials.thisN'
}
for old_name, new_name in rename_map.items():
    if old_name in big_dataframe.columns:
        big_dataframe.rename(columns={old_name: new_name}, inplace=True)

# Debug print
print("\nColumns in big_dataframe:", big_dataframe.columns)
print("Sample rows:\n", big_dataframe.head())

# Save the big DataFrame for reference
big_dataframe.to_csv(os.path.join(data_directory, 'big_dataframe.csv'), index=False)
print("\nSaved 'big_dataframe.csv'.")

###############################################################################
# 3) OPTIONAL: CLASSES FOR SIMULATION (IF NEEDED)
###############################################################################
class RewardLearner:
    """
    Simple Rescorla-Wagner:
      R_hat[t] = R_hat[t-1] + alpha_r * (R_t - R_hat[t-1])
    """
    def __init__(self, alpha_r=0.1, init_reward=3.5, sigma_r=0.7):
        self.alpha_r = alpha_r
        self.R_hat = init_reward
        self.sigma_r = sigma_r

    def update(self, received_reward):
        delta_r = received_reward - self.R_hat
        self.R_hat += self.alpha_r * delta_r
        return self.R_hat

    def belief_distribution(self):
        return norm(loc=self.R_hat, scale=self.sigma_r)


class EffortLearner:
    """
    Direct threshold feedback:
      E_hat[t+1] = E_hat[t] + alpha_e * (threshold[t] - E_hat[t])
    """
    def __init__(self, alpha_e=0.1, init_effort=50.0):
        self.alpha_e = alpha_e
        self.E_hat = init_effort

    def update(self, true_threshold):
        delta_e = true_threshold - self.E_hat
        self.E_hat += self.alpha_e * delta_e
        self.E_hat = np.clip(self.E_hat, 0, 100)
        return self.E_hat


class EffortDiscounter:
    """
    We search effort e in [0..100], compute:
      p_success(e) = norm.cdf((e - E_hat)/sigma)
      SV(e) = R_hat - kappa*e
      EU(e) = SV(e)*p_success(e)
    Then either argmax or softmax policy.
    """
    def __init__(self, kappa=0.05, sigma=5.0, beta=5.0, policy='argmax'):
        self.kappa = kappa
        self.sigma = sigma
        self.beta = beta
        self.policy = policy

    def choose_effort(self, R_hat, E_hat):
        e_values = np.arange(101)  # integer efforts from 0..100
        EU_values = []
        for e in e_values:
            p_success = norm.cdf((e - E_hat)/self.sigma)
            SV = R_hat - self.kappa * e
            EU_values.append(SV * p_success)

        EU_values = np.array(EU_values)
        if self.policy == 'argmax':
            chosen_idx = np.argmax(EU_values)
            return e_values[chosen_idx]
        else:
            # softmax
            max_eu = np.max(EU_values)
            exp_vals = np.exp(self.beta * (EU_values - max_eu))
            probs = exp_vals / np.sum(exp_vals)
            return np.random.choice(e_values, p=probs)


###############################################################################
# 4) SIMULATION FUNCTION (OPTIONAL)
###############################################################################
def simulate_experiment(num_trials=60,
                        reward_learner_params=None,
                        effort_learner_params=None,
                        discounter_params=None,
                        data=None,
                        ignore_zero_reward=True,
                        normalize_reward=False,
                        normalize_effort=False):
    """
    Example simulation to show how you'd step through an experiment.
    This is optional if you already have real data.
    """
    if reward_learner_params is None:
        reward_learner_params = {}
    if effort_learner_params is None:
        effort_learner_params = {}
    if discounter_params is None:
        discounter_params = {}

    if normalize_reward and (data is not None):
        if ignore_zero_reward:
            data['reward'] = data['reward'].replace(0, np.nan)
        min_r = data['reward'].min()
        max_r = data['reward'].max()
        data['reward'] = (data['reward'] - min_r) / (max_r - min_r)

    if normalize_effort and (data is not None):
        data['threshold'] = normalize_column_to_0_100(data, 'threshold')

    # Initialize model classes
    RL = RewardLearner(**reward_learner_params)
    EL = EffortLearner(**effort_learner_params)
    discounter = EffortDiscounter(**discounter_params)

    results = {
        'trial': [],
        'threshold': [],
        'reward': [],
        'chosen_effort': [],
        'outcome': [],
        'R_hat': [],
        'E_hat': []
    }

    if data is None:
        print("No data provided to simulate_experiment. Exiting.")
        return pd.DataFrame(results)

    actual_num_trials = min(num_trials, len(data))
    for t in range(actual_num_trials):
        threshold_t = data.iloc[t]['threshold']
        reward_t = data.iloc[t]['reward']

        if ignore_zero_reward and reward_t == 0:
            print(f"Trial {t+1}: Skipping zero-reward trial")
            continue

        # Current estimates
        R_hat_t = RL.R_hat
        E_hat_t = EL.E_hat

        chosen_e = discounter.choose_effort(R_hat_t, E_hat_t)
        p_success = norm.cdf((chosen_e - E_hat_t) / discounter.sigma)
        outcome = np.random.binomial(1, p_success)
        received_reward = reward_t if outcome == 1 else 0.0

        RL.update(received_reward)
        EL.update(threshold_t)

        results['trial'].append(t+1)
        results['threshold'].append(threshold_t)
        results['reward'].append(reward_t)
        results['chosen_effort'].append(chosen_e)
        results['outcome'].append(outcome)
        results['R_hat'].append(RL.R_hat)
        results['E_hat'].append(EL.E_hat)

    return pd.DataFrame(results)


###############################################################################
# 5) PARAMETER RECOVERY / FITTING
###############################################################################
def model_nll(params, subdf, fixed_sigma=10.0):
    """
    Computes the negative log-likelihood under the effort–reward model
    for one participant's data, given parameters.

    params = [alpha_r, alpha_e, kappa, beta]
      alpha_r : reward learning rate in [0..1]
      alpha_e : effort learning rate in [0..1]
      kappa   : cost rate per unit effort in [0..(some range)]
      beta    : inverse temperature for softmax

    subdf: the participant's data (one ID), containing columns:
           'trial', 'threshold', 'reward', 'chosen_effort', (optionally 'outcome')

    fixed_sigma: we keep sigma constant for all participants in this example.
    Returns: nLL (float) = negative log-likelihood
    """
    alpha_r, alpha_e, kappa, beta = params
    sigma = fixed_sigma

    # Initialize hidden states
    R_hat = 3.0   # near midpoint if reward ~1..7
    E_hat = 50.0  # near midpoint if threshold ~0..100

    # Sort by trial to ensure correct chronological order
    subdf = subdf.sort_values('trial', ascending=True)

    nLL = 0.0
    eps = 1e-12

    for _, row in subdf.iterrows():
        threshold_t = row['threshold']
        reward_t = row['reward']
        chosenEff_t = row['chosen_effort']

        # All possible efforts in [0..100]
        e_values = np.arange(101)
        EU = np.zeros_like(e_values, dtype=float)

        for iE, e_test in enumerate(e_values):
            p_succ = norm.cdf((e_test - E_hat) / sigma)
            SV = R_hat - kappa * e_test
            EU[iE] = SV * p_succ

        # Softmax over EU
        EU_max = np.max(EU)
        expEU = np.exp(beta * (EU - EU_max))
        p_e = expEU / (np.sum(expEU) + eps)

        # Probability of the observed chosen effort
        if chosenEff_t in e_values:
            p_chosen = p_e[int(chosenEff_t)]
        else:
            # If for some reason chosenEff_t isn't an integer in [0..100], clamp
            p_chosen = eps

        nLL -= np.log(max(p_chosen, eps))

        # Check if an actual outcome is in the data
        if 'outcome' in subdf.columns:
            outcome = row['outcome']
        else:
            # Or simulate outcome from the model
            p_succ_obs = norm.cdf((chosenEff_t - E_hat) / sigma)
            outcome = 1 if (np.random.rand() < p_succ_obs) else 0

        # The participant receives 'reward_t' only if outcome==1
        received_reward = reward_t if outcome == 1 else 0.0

        # Update the learners
        delta_r = received_reward - R_hat
        R_hat = R_hat + alpha_r * delta_r

        delta_e = threshold_t - E_hat
        E_hat = E_hat + alpha_e * delta_e
        E_hat = np.clip(E_hat, 0, 100)

    return nLL


def fit_participant(subdf, init_params=None, bounds=None, fixed_sigma=10.0):
    """
    Minimizes negative log-likelihood for one participant with a fixed sigma.
    subdf: DataFrame with that participant's data
    init_params: initial guess [alpha_r, alpha_e, kappa, beta]
    bounds: list of (min, max) for each parameter
    fixed_sigma: constant sigma used in model_nll
    """
    if init_params is None:
        init_params = [0.3, 0.3, 0.05, 1.0]  # example guess
    if bounds is None:
        # alpha_r in [0,1], alpha_e in [0,1], kappa in [0,1], beta in [0,10]
        bounds = [(0,1), (0,1), (0,1), (0,10)]

    # Objective function for minimization
    def objective(x):
        return model_nll(x, subdf, fixed_sigma=fixed_sigma)

    res = minimize(
        fun=objective,
        x0=init_params,
        bounds=bounds,
        method='L-BFGS-B',
        options={'maxfun': 10000, 'disp': False}
    )
    return res.x, res.fun


###############################################################################
# 6) MAIN: EXAMPLE PARAMETER RECOVERY
###############################################################################
def main():
    # Use the big_dataframe loaded at the top (contains all participants).
    df = big_dataframe
    unique_ids = df['id'].unique()

    # We'll store the fit results here
    results_list = []

    # We'll pick a fixed sigma for all participants for now
    fixed_sigma_value = 10.0  # or e.g. 5.0, or 0.1, etc.

    for pid in unique_ids:
        subdf = df[df['id'] == pid].copy()
        
        # Fit the participant
        best_params, best_nll = fit_participant(
            subdf=subdf,
            init_params=[0.3, 0.3, 0.05, 1.0],
            bounds=[(0,1),(0,1),(0,1),(0,10)],
            fixed_sigma=fixed_sigma_value
        )
        
        alpha_r, alpha_e, kappa, beta = best_params
        results_list.append({
            'id': pid,
            'alpha_r': alpha_r,
            'alpha_e': alpha_e,
            'kappa': kappa,
            'beta': beta,
            'nLL': best_nll
        })

    # Convert results to DataFrame and save
    results_df = pd.DataFrame(results_list)
    print("\nParameter Fit Results:\n", results_df)

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    out_filename = f"fit_results_fixedSigma_{timestamp}.csv"
    results_df.to_csv(out_filename, index=False)
    print(f"Saved parameter fit results to {out_filename}")
    plot_per_participant(big_dataframe, results_df, fixed_sigma=10.0)

    # Plot parameter distributions
    params = ['alpha_r','alpha_e','kappa','beta']
    plt.figure(figsize=(10, 6))
    for i, p in enumerate(params, start=1):
        plt.subplot(2, 2, i)
        sns.histplot(data=results_df, x=p, kde=True)
        plt.title(f"Distribution of {p}")
    plt.tight_layout()
    plt.show()


# Run main automatically (comment out if you prefer to call main() manually)
if __name__ == "__main__":
    main()


Loaded task_data_nobehav.csv (renamed to 'reward' and 'threshold').
Reading CSV file: sub-66a8029007c122fe8086afcb_rewardeffortlearning_2024-09-06_11h17.22.698.csv
Skipping file: sub-6547bf8012d4702680d55663_rewardeffortlearning_2024-09-06_10h30.03.678.log.gz
Reading CSV file: sub-55bd8669fdf99b5bfc7d4cfc_rewardeffortlearning_2024-09-06_11h25.07.641.csv
Reading CSV file: sub-63474e67a5fd298c6103c409_rewardeffortlearning_2024-09-06_10h50.53.039.csv
Reading CSV file: sub-5cba9a8a214b1a0016ccf708_rewardeffortlearning_2024-09-06_08h21.17.302.csv
Skipping file: sub-601b28841ca7bb2f8edaa293_rewardeffortlearning_2024-09-06_11h16.31.372.log.gz
Skipping file: PARTICIPANT_RewardEffortLearning_pilot2_2024-09-06_11h56.00.479.log.gz
Skipping file: .DS_Store
Skipping file: PARTICIPANT_RewardEffortLearning_pilot2_2024-09-06_11h53.27.161.csv
Skipping file: sub-66706a23c37c6099480f039d_rewardeffortlearning_2024-09-06_11h17.32.305.log.gz
Skipping file: sub-600319e3a3b1a337cab57a3e_rewardeffortlearning_2

KeyError: 'threshold'

<Figure size 800x500 with 0 Axes>